<a href="https://colab.research.google.com/github/thiagosdpaiva/CNU-2-Discursiva-Extract/blob/main/Extra%C3%A7%C3%A3o_Bloco_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install pymupdf

import fitz
from google.colab import files
import re

# ================= CONFIGURAÇÃO =================
PAGINA_INICIO = 110
GATILHO_INICIO = "Arquitetura"

PAGINA_FIM = 163
FREIO_FIM = "Bloco"

IGNORAR_TOPO = 50
IGNORAR_BASE = 50
# ================================================

def extrair_dados_pdf():
    print("Selecione o PDF no botão abaixo:")
    uploaded = files.upload()

    if not uploaded:
        print("Nenhum arquivo enviado.")
        return None

    nome_arquivo = next(iter(uploaded))
    texto_recortado = ""
    idx_inicio = PAGINA_INICIO - 1
    idx_fim = PAGINA_FIM - 1

    # --- ETAPA 1: LEITURA DO PDF ---
    try:
        with fitz.open(nome_arquivo) as doc:
            for i in range(idx_inicio, idx_fim + 1):
                pagina = doc[i]
                rect = pagina.rect
                rect.y0 += IGNORAR_TOPO
                rect.y1 -= IGNORAR_BASE

                if rect.y0 < rect.y1:
                    texto_pag = pagina.get_text(clip=rect)
                else:
                    texto_pag = ""

                if i == idx_inicio and i == idx_fim:
                    if GATILHO_INICIO in texto_pag and FREIO_FIM in texto_pag:
                        p1 = texto_pag.split(GATILHO_INICIO, 1)[1]
                        texto_recortado += p1.split(FREIO_FIM, 1)[0]
                elif i == idx_inicio:
                    if GATILHO_INICIO in texto_pag:
                        texto_recortado += texto_pag.split(GATILHO_INICIO, 1)[1]
                elif i == idx_fim:
                    if FREIO_FIM in texto_pag:
                        texto_recortado += texto_pag.split(FREIO_FIM, 1)[0]
                    else:
                        texto_recortado += texto_pag
                else:
                    texto_recortado += texto_pag

    except Exception as e:
        return f"Erro na leitura: {e}"

    # --- ETAPA 2: EXTRAÇÃO POR VETOR (CARGO) ---
    print("\nProcessando dados...")
    texto_limpo = texto_recortado.replace('\n', ' ')
    partes = re.split(r"\((.*?)\)", texto_limpo)

    # Dicionário que armazena vetores separados
    banco_de_vetores = {}

    for i in range(1, len(partes), 2):
        nome_vetor = partes[i].strip()
        conteudo_bruto = partes[i+1]

        # Pega números como strings
        lista_ids = re.findall(r"\d+", conteudo_bruto)

        # Armazena no dicionário separado por cargo
        banco_de_vetores[nome_vetor] = lista_ids

    return banco_de_vetores

# --- EXECUÇÃO PRINCIPAL ---
# Aqui recuperamos o dicionário com tudo separado
dados_separados = extrair_dados_pdf()

if isinstance(dados_separados, dict):

    # ---------------------------------------------------------
    # PARTE 1: MOSTRAR OS DADOS SEPARADOS (SEM ALTERAR NADA)
    # ---------------------------------------------------------
    print("\n" + "="*60)
    print(f"{'RELATÓRIO 1: CANDIDATOS POR CARGO (VETORES INDIVIDUAIS)':^60}")
    print("="*60)
    print(f"{'NOME DO VETOR':<30} | {'QTD':<10}")
    print("-" * 60)

    for nome, lista in dados_separados.items():
        print(f"{nome:<30} | {len(lista):<10}")
        # A lista 'lista' continua existindo aqui para cada cargo

    print("-" * 60)

    # ---------------------------------------------------------
    # PARTE 2: CRIAR O VETOR UNIFICADO (NOVA FUNCIONALIDADE)
    # ---------------------------------------------------------
    conjunto_temp = set()

    # Percorremos o dicionário original para alimentar o conjunto
    for lista in dados_separados.values():
        conjunto_temp.update(lista)

    # Cria a lista final e limpa
    Candidatos_Bloco4 = sorted(list(conjunto_temp))

    print("\n" + "="*60)
    print(f"{'RELATÓRIO 2: CONSOLIDAÇÃO FINAL (Candidatos_Bloco4)':^60}")
    print("="*60)

    qtd_final = len(Candidatos_Bloco4)

    # Amostra visual
    if qtd_final > 10:
        amostra = f"{Candidatos_Bloco4[:5]} ... {Candidatos_Bloco4[-5:]}"
    else:
        amostra = str(Candidatos_Bloco4)

    print(f"Vetor Unificado: Candidatos_Bloco4")
    print(f"Total de Elementos Únicos: {qtd_final}")
    print("-" * 60)
    print(f"Amostra: {amostra}")
    print("="*60)

else:
    print(dados_separados)

Selecione o PDF no botão abaixo:


Saving editalconvocacao_provadiscursiva_retificado.pdf to editalconvocacao_provadiscursiva_retificado (1).pdf

Processando dados...

  RELATÓRIO 1: CANDIDATOS POR CARGO (VETORES INDIVIDUAIS)   
NOME DO VETOR                  | QTD       
------------------------------------------------------------
B4-01-A                        | 283       
B4-02-A                        | 189       
B4-02-B                        | 144       
B4-02-C                        | 286       
B4-02-D                        | 431       
B4-02-E                        | 371       
B4-02-F                        | 328       
B4-02-G                        | 223       
B4-02-H                        | 120       
B4-02-I                        | 222       
B4-02-J                        | 187       
B4-02-K                        | 193       
B4-02-L                        | 276       
B4-02-M                        | 246       
B4-02-N                        | 319       
B4-02-O                        | 446     

In [13]:
# --- CÉLULA 2: CARREGAMENTO EXTERNO, MAPEAMENTO E AGRUPAMENTO ---
!pip install pymupdf pandas

import fitz  # PyMuPDF
import re
import pandas as pd
from google.colab import files

# ================= CONFIGURAÇÃO =================
# Ajuste as páginas conforme seu Edital
PAGINA_INICIO_EDITAL = 7
PAGINA_FIM_EDITAL = 104

PADRAO_CODIGO = r"[A-Z]\d-\d{2}-[A-Z]"

ESPECIALIDADES_ALVO = [
    "Engenharia Mecânica", "Engenharia Aeronáutica", "Engenharia Aeroespacial",
    "Engenharia Eletrônica", "Engenharia Elétrica", "Engenharia de Sistemas",
    "Engenharia Mecatrônica", "Engenharia de Controle e Automação",
    "Engenharia de Telecomunicações", "Engenharia Civil", "Engenharia Naval",
    "Engenharia de Produção", "Engenharia de Materiais", "Engenharia Florestal",
    "Engenharia Metalúrgica", "Engenharia Cartográfica", "Engenharia Agronômica",
    "Engenharia de Pesca", "Engenharia Ambiental", "Engenharia",
    "qualquer área de Engenharia", "Mecânica", "Mecatrônica", "Telecomunicação",
    "Telecomunicações", "Eletrônica", "Automação", "Instrumentação", "Agrimensura",
    "Agronomia", "Arquitetura e Urbanismo", "Arquitetura", "qualquer área do conhecimento"
]
# ================================================

def carregar_e_mapear_agrupado():
    print(">>> AÇÃO NECESSÁRIA: Faça o upload do PDF do EDITAL:")
    uploaded = files.upload()
    if not uploaded: return None

    nome_arquivo = next(iter(uploaded))
    print(f"\nLendo: '{nome_arquivo}'...")

    texto_completo = ""
    try:
        with fitz.open(nome_arquivo) as doc:
            inicio = max(0, PAGINA_INICIO_EDITAL - 1)
            fim = min(len(doc), PAGINA_FIM_EDITAL)
            for i in range(inicio, fim):
                texto_completo += doc[i].get_text() + "\n"
    except Exception as e:
        print(f"Erro: {e}")
        return None

    # --- PROCESSAMENTO ---
    texto_limpo = texto_completo.replace('\n', ' ')
    ocorrencias = list(re.finditer(PADRAO_CODIGO, texto_limpo))

    lista_relacoes = []
    print(f"Analisando requisitos de {len(ocorrencias)} cargos...")

    for i in range(len(ocorrencias)):
        match = ocorrencias[i]
        codigo = match.group()

        inicio = match.end()
        fim = ocorrencias[i+1].start() if i + 1 < len(ocorrencias) else len(texto_limpo)

        desc_lower = texto_limpo[inicio:fim].lower()

        # Varre TODAS as especialidades. Se achar 3, adiciona 3 linhas.
        encontrou_alguma = False
        for esp in ESPECIALIDADES_ALVO:
            if esp.lower() in desc_lower:
                lista_relacoes.append({"Cargo": codigo, "Aceita": esp})
                encontrou_alguma = True

        if "qualquer área de formação" in desc_lower and not encontrou_alguma:
             lista_relacoes.append({"Cargo": codigo, "Aceita": "Qualquer área de formação"})

    return pd.DataFrame(lista_relacoes)

# --- EXECUÇÃO ---
df_mapa = carregar_e_mapear_agrupado()

if df_mapa is not None and not df_mapa.empty:
    # 1. Cria o Relatório Agrupado (Visualização Amigável)
    # Agrupa por Cargo e junta as especialidades com uma vírgula
    df_agrupado = df_mapa.groupby('Cargo')['Aceita'].apply(lambda x: ', '.join(sorted(set(x)))).reset_index()

    print("\n" + "="*80)
    print(f"{'RELATÓRIO CONSOLIDADO: CURSOS ACEITOS POR CÓDIGO':^80}")
    print("="*80)

    # Configura o pandas para mostrar o texto todo da coluna, sem cortar (...)
    pd.set_option('display.max_colwidth', None)

    # Mostra os primeiros 20 cargos
    print(df_agrupado.head(20))

    print("\n" + "-"*80)
    print(f"Total de Cargos Mapeados: {len(df_agrupado)}")

    # Salva o arquivo CSV detalhado (para usar no Excel)
    df_agrupado.to_csv("cargos_cursos_agrupados.csv", index=False)
    print("Arquivo 'cargos_cursos_agrupados.csv' pronto para download.")

else:
    print("Nenhum dado encontrado.")

>>> AÇÃO NECESSÁRIA: Faça o upload do PDF do EDITAL:


Saving cpnu2_anexo_blocotematico4_8aretif.pdf to cpnu2_anexo_blocotematico4_8aretif (3).pdf

Lendo: 'cpnu2_anexo_blocotematico4_8aretif (3).pdf'...
Analisando requisitos de 88 cargos...

                RELATÓRIO CONSOLIDADO: CURSOS ACEITOS POR CÓDIGO                
      Cargo  \
0   B4-01-A   
1   B4-02-A   
2   B4-02-B   
3   B4-02-C   
4   B4-02-D   
5   B4-02-E   
6   B4-02-F   
7   B4-02-G   
8   B4-02-H   
9   B4-02-I   
10  B4-02-J   
11  B4-02-K   
12  B4-02-L   
13  B4-02-M   
14  B4-02-N   
15  B4-02-O   
16  B4-03-A   
17  B4-03-B   
18  B4-03-C   
19  B4-03-D   

                                                                                                                                                                                                                                       Aceita  
0                                                                                                                                                                               

In [14]:
# --- CÉLULA 3: INTERFACE INTERATIVA (COM LISTA DE ÚNICOS) ---
import ipywidgets as widgets
from IPython.display import display, clear_output
import os

# 1. Verificação de segurança
if 'df_mapa' not in globals() or 'dados_separados' not in globals():
    print("⚠️ ERRO: Por favor, execute as Células 1 e 2 antes desta.")
else:
    # 2. Prepara a lista de opções
    opcoes_disponiveis = sorted(df_mapa['Aceita'].unique())

    # 3. Criação dos Widgets
    print("SELECIONE AS ESPECIALIDADES PARA FILTRAR:")

    checkboxes = [
        widgets.Checkbox(value=False, description=op, indent=False)
        for op in opcoes_disponiveis
    ]

    grid = widgets.GridBox(
        checkboxes,
        layout=widgets.Layout(grid_template_columns="repeat(3, 300px)")
    )

    botao_filtrar = widgets.Button(
        description="CALCULAR ÚNICOS",
        button_style='success', # Cor verde
        layout=widgets.Layout(width='90%', height='50px', margin='20px 0px 0px 0px'),
        icon='check'
    )

    output_area = widgets.Output()

    # 4. Função Lógica
    def ao_clicar_no_botao(b):
        with output_area:
            clear_output()

            selecionados = [box.description for box in checkboxes if box.value]

            if not selecionados:
                print("⚠️ Nenhuma especialidade selecionada.")
                return

            # Exibe o que foi selecionado
            nomes_exibicao = ", ".join(selecionados[:5])
            if len(selecionados) > 5: nomes_exibicao += "..."
            print(f"🔎 Filtro: {nomes_exibicao}\n")

            # A) Achar Cargos
            cargos_alvo = df_mapa[df_mapa['Aceita'].isin(selecionados)]['Cargo'].unique()

            # B) Buscar Candidatos e Unificar
            total_inscricoes = 0
            vetores_encontrados = 0

            # O 'set' é o segredo: ele não permite números repetidos
            conjunto_unicos = set()

            print(f"{'CÓDIGO':<15} | {'INSCRIÇÕES':<12} | {'STATUS'}")
            print("-" * 60)

            for cargo in cargos_alvo:
                cargo_limpo = cargo.strip()

                if cargo_limpo in dados_separados:
                    lista = dados_separados[cargo_limpo]
                    qtd = len(lista)

                    total_inscricoes += qtd
                    vetores_encontrados += 1

                    # Adiciona os números ao conjunto (duplicatas são ignoradas aqui)
                    conjunto_unicos.update(lista)

                    print(f"{cargo_limpo:<15} | {qtd:<12} | Localizado")
                else:
                    print(f"{cargo_limpo:<15} | {0:<12} | Vazio/Não achado")

            # Converte o conjunto para uma lista ordenada para exibição/salvamento
            lista_final_unica = sorted(list(conjunto_unicos))
            qtd_unicos = len(lista_final_unica)

            print("="*60)
            print(f"RESUMO FINAL:")
            print(f" -> Especialidades Selecionadas: {len(selecionados)}")
            print(f" -> Cargos Analisados: {len(cargos_alvo)}")
            print(f" -> Soma de Inscrições (Bruto): {total_inscricoes}")
            print("-" * 60)
            print(f" -> CANDIDATOS ÚNICOS (SEM REPETIÇÃO): {qtd_unicos}")
            print("="*60)

            # Mostra uma amostra na tela
            if qtd_unicos > 0:
                print(f"Amostra dos únicos: {lista_final_unica[:10]} ...")

                # Salva em arquivo para o usuário baixar
                nome_arquivo = "lista_candidatos_unicos.txt"
                with open(nome_arquivo, "w") as f:
                    f.write("\n".join(lista_final_unica))

                print(f"\n✅ Arquivo '{nome_arquivo}' gerado com sucesso!")
                print("Você pode baixá-lo na aba de arquivos à esquerda.")

    botao_filtrar.on_click(ao_clicar_no_botao)
    display(grid, botao_filtrar, output_area)

SELECIONE AS ESPECIALIDADES PARA FILTRAR:


GridBox(children=(Checkbox(value=False, description='Agrimensura', indent=False), Checkbox(value=False, descri…

Button(button_style='success', description='CALCULAR ÚNICOS', icon='check', layout=Layout(height='50px', margi…

Output()

In [15]:
# --- CÉLULA 4: SELEÇÃO DIRETA DE CÓDIGOS DE VAGA ---
import ipywidgets as widgets
from IPython.display import display, clear_output

# 1. Verificação de segurança
if 'dados_separados' not in globals():
    print("⚠️ ERRO: A variável 'dados_separados' não existe.")
    print("Por favor, rode a Célula 1 (Extração de Candidatos) primeiro.")
else:
    # 2. Pega todas as chaves (códigos) que foram extraídas na Célula 1
    lista_codigos_disponiveis = sorted(dados_separados.keys())

    if not lista_codigos_disponiveis:
        print("⚠️ Nenhum código de vaga foi encontrado na extração anterior.")
    else:
        print(f"Foram encontrados {len(lista_codigos_disponiveis)} códigos de vaga extraídos.")
        print("SELECIONE OS CÓDIGOS PARA CALCULAR O TOTAL DE ÚNICOS:")

        # 3. Criação dos Checkboxes
        # Usamos uma lista de checkboxes para você poder marcar visualmente
        checkboxes_vagas = [
            widgets.Checkbox(value=False, description=cod, indent=False)
            for cod in lista_codigos_disponiveis
        ]

        # Organiza em Grid (4 colunas para caber mais códigos na tela)
        grid_vagas = widgets.GridBox(
            checkboxes_vagas,
            layout=widgets.Layout(grid_template_columns="repeat(4, 250px)")
        )

        # Botão de Cálculo
        botao_calc_vagas = widgets.Button(
            description="SOMAR SELECIONADOS",
            button_style='primary', # Cor azul escuro
            layout=widgets.Layout(width='95%', height='50px', margin='20px 0px 0px 0px'),
            icon='calculator'
        )

        output_vagas = widgets.Output()

        # 4. Função Lógica
        def ao_clicar_somar(b):
            with output_vagas:
                clear_output()

                # Identifica quais caixas estão marcadas
                codigos_escolhidos = [box.description for box in checkboxes_vagas if box.value]

                if not codigos_escolhidos:
                    print("⚠️ Nenhum código selecionado. Marque pelo menos um.")
                    return

                print(f"🔎 Analisando {len(codigos_escolhidos)} vetores selecionados...\n")

                conjunto_unicos = set()
                total_bruto = 0

                print(f"{'CÓDIGO':<15} | {'INSCRITOS':<10}")
                print("-" * 30)

                for codigo in codigos_escolhidos:
                    # Recupera a lista de candidatos direto da memória
                    lista_candidatos = dados_separados[codigo]
                    qtd = len(lista_candidatos)

                    # Atualiza contadores
                    total_bruto += qtd
                    conjunto_unicos.update(lista_candidatos)

                    print(f"{codigo:<15} | {qtd:<10}")

                # Prepara resultado final
                lista_final = sorted(list(conjunto_unicos))
                qtd_unicos = len(lista_final)

                print("="*60)
                print("RESULTADO DA SELEÇÃO:")
                print(f" -> Soma simples de inscrições: {total_bruto}")
                print("-" * 60)
                print(f" -> CANDIDATOS ÚNICOS (SEM DUPLICATAS): {qtd_unicos}")
                print("="*60)

                # Gera arquivo para download
                if qtd_unicos > 0:
                    nome_arquivo = "candidatos_por_codigo_selecionado.txt"
                    with open(nome_arquivo, "w") as f:
                        f.write("\n".join(lista_final))
                    print(f"\n✅ Lista salva em '{nome_arquivo}'")

        # Conecta botão
        botao_calc_vagas.on_click(ao_clicar_somar)

        # Exibe interface
        display(grid_vagas, botao_calc_vagas, output_vagas)

Foram encontrados 87 códigos de vaga extraídos.
SELECIONE OS CÓDIGOS PARA CALCULAR O TOTAL DE ÚNICOS:


GridBox(children=(Checkbox(value=False, description='B4-01-A', indent=False), Checkbox(value=False, descriptio…

Button(button_style='primary', description='SOMAR SELECIONADOS', icon='calculator', layout=Layout(height='50px…

Output()